# Racing Statistic Analysis

### TODO

- 샘플링 방식을 달리 해서 말을 구체적으로 발라낼 것
- 가치주처럼 다크호스 말을 찾아낼 것
- 전체가 아닌 우승마를 맞추는걸로 변경해야 한다
- 실제 Testset에 적용하여 결과를 보자

In [1]:
import numpy as np
import pandas as pd

# Load Data

In [2]:
train = pd.read_csv("../Dataset/hr_train.csv")
test = pd.read_csv("../Dataset/hr_test.csv")
train = train.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)
train.head(3)

,rcDate,meet,rcNo,ord,rcTime,age,chulNo,corner_txt,differ,hrName,...,owName,plc,prdCtyNm,sex,trName,wgBudam,wgHr,win,year,isWin
0,2000-01-30,서울,1,12,01:22:01,5,3,- - - 12 - 11 -,2½,서천,...,어울림네트웍스,5,뉴,암,김문갑,52.0,391.0,52,2000,False
1,2000-01-30,서울,1,8,01:21:01,4,4,- - - 8 - 8 -,½,런패스터,...,성환옥,2,미,암,최혜식,52.0,423.0,17,2000,False
2,2000-01-30,서울,1,10,01:22:00,4,5,- - - 11 - 12 -,¾,한가람,...,이영욱a,24,뉴,암,최영주,52.0,442.0,732,2000,False


# Data Sampling
- 서울, 부경마 / 제주마
- 단승식 / 복승식

In [17]:
from datetime import datetime
from datetime import timedelta
import re

In [6]:
cols_to_drop = ['chulNo', 'ord', 'rcNo', 'rcTime', 'differ', 'corner_txt', 'year']
train = train.drop(cols_to_drop, axis=1)
train = train[train['meet']=='서울']
train.head()

,rcDate,meet,age,hrName,jkName,owName,plc,prdCtyNm,sex,trName,wgBudam,wgHr,win,isWin
0,2000-01-30,서울,5,서천,박동랑,어울림네트웍스,5,뉴,암,김문갑,52.0,391.0,52,False
1,2000-01-30,서울,4,런패스터,김동균,성환옥,2,미,암,최혜식,52.0,423.0,17,False
2,2000-01-30,서울,4,한가람,이정선,이영욱a,24,뉴,암,최영주,52.0,442.0,732,False
3,2000-01-30,서울,6,캐터리나블루,김효섭,이종덕,8,미,암,이인호,54.0,424.0,98,False
4,2000-01-30,서울,3,데이빗슨,한유영,한병규,4,뉴,거,한일영,55.0,487.0,57,False


## One-Hot Encoding
- prdCtyNm : 경주마 생산국가명 / 호, 한 등 17개국
- sex : 말의성별 / 암, 거, 수
- hrName : 말의이름 / 웨스턴타이틀, 두루미 등
- jkName : 경주마 기승하는 기수명
- owName : 경주마 소유한 마주명
- trName : 조교사명

In [10]:
cols_to_transform = ['sex']
train = pd.get_dummies(train, columns=cols_to_transform)
train.head(3)

,rcDate,meet,age,hrName,jkName,owName,plc,prdCtyNm,trName,wgBudam,wgHr,win,isWin,sex_거,sex_수,sex_암
1072,2000-01-30,제주,3,남이장군,이재웅,강상우,32,한,김영래,52.0,204.0,21,False,0,1,0
1073,2000-01-30,제주,3,동부로,김기환,한성택,1,한,임용찬,55.0,238.0,2,False,0,1,0
1074,2000-01-30,제주,3,쓰리쎄븐,심도연,김덕인,1,한,박석천,54.0,216.0,2,False,0,0,1


# Modeling

In [12]:
predictors = ["plc", "win"]
x = train[predictors]
y = train["isWin"]

## DecisionTree & RandomForest

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [14]:
decision = DecisionTreeClassifier().fit(x,y)
forest = RandomForestClassifier(n_jobs=-1).fit(x,y)
print(forest)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)


## Cross Validation

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [17]:
decision_score = cross_val_score(decision, x, y, cv=5).mean()
forst_score = cross_val_score(forest, x, y, cv=5).mean()

print("DecisionTree = {0:.6f}".format(decision_score))
print("RandomForest = {0:.6f}".format(forst_score))

DecisionTree = 0.899604
RandomForest = 0.899716


# Result
- plc, win: 0.899656
- age, sex: 0.895907